In [46]:
import gurobipy as gp

In [47]:
# Dados brutos
qtd_fabricas = 4
qtd_clientes = 9
lista_ofertas = [140, 160, 200, 190]
lista_demandas = [50, 80, 30, 50, 100, 90, 20, 70, 120]
custos_transporte = [
    [12, 17, 35, 28],
    [25, 26, 15, 30],
    [39, 20, 18, 37],
    [17, 25, 20, 30],
    [38, 30, 12, 28],
    [40, 25, 42, 36],
    [8, 14, 27, 16],
    [25, 20, 26, 24],
    [13, 15, 19, 32]]

print(custos_transporte)

[[12, 17, 35, 28], [25, 26, 15, 30], [39, 20, 18, 37], [17, 25, 20, 30], [38, 30, 12, 28], [40, 25, 42, 36], [8, 14, 27, 16], [25, 20, 26, 24], [13, 15, 19, 32]]


In [48]:
fabricas = ["Fabricas_{}".format(i + 1)for i in range(qtd_fabricas)]
clientes = ["Cliente_{}".format(i + 1) for i in range(qtd_clientes)]


In [49]:
ofertas = dict()

for i in range(qtd_fabricas):
    ofertas[fabricas[i]] = lista_ofertas[i]

demandas = dict()
for i in range(qtd_clientes):
    demandas[clientes[i]] = lista_demandas[i]
    
custos = dict()
for i in range(qtd_fabricas):
    for j in range(qtd_clientes):
        custos[fabricas[i],clientes[j]] = custos_transporte[j][i]
print(custos)

{('Fabricas_1', 'Cliente_1'): 12, ('Fabricas_1', 'Cliente_2'): 25, ('Fabricas_1', 'Cliente_3'): 39, ('Fabricas_1', 'Cliente_4'): 17, ('Fabricas_1', 'Cliente_5'): 38, ('Fabricas_1', 'Cliente_6'): 40, ('Fabricas_1', 'Cliente_7'): 8, ('Fabricas_1', 'Cliente_8'): 25, ('Fabricas_1', 'Cliente_9'): 13, ('Fabricas_2', 'Cliente_1'): 17, ('Fabricas_2', 'Cliente_2'): 26, ('Fabricas_2', 'Cliente_3'): 20, ('Fabricas_2', 'Cliente_4'): 25, ('Fabricas_2', 'Cliente_5'): 30, ('Fabricas_2', 'Cliente_6'): 25, ('Fabricas_2', 'Cliente_7'): 14, ('Fabricas_2', 'Cliente_8'): 20, ('Fabricas_2', 'Cliente_9'): 15, ('Fabricas_3', 'Cliente_1'): 35, ('Fabricas_3', 'Cliente_2'): 15, ('Fabricas_3', 'Cliente_3'): 18, ('Fabricas_3', 'Cliente_4'): 20, ('Fabricas_3', 'Cliente_5'): 12, ('Fabricas_3', 'Cliente_6'): 42, ('Fabricas_3', 'Cliente_7'): 27, ('Fabricas_3', 'Cliente_8'): 26, ('Fabricas_3', 'Cliente_9'): 19, ('Fabricas_4', 'Cliente_1'): 28, ('Fabricas_4', 'Cliente_2'): 30, ('Fabricas_4', 'Cliente_3'): 37, ('Fabricas

In [50]:
m = gp.Model()

#
transporte = m.addVars(fabricas, clientes)

#Restricoes de Offerta
c_ofertas = m.addConstrs(gp.quicksum(transporte[i,j] for j in clientes) <= ofertas[i] for i in fabricas)
#Restricoes de Demandas
c_demandas = m.addConstrs(gp.quicksum(transporte[i,j] for i in fabricas) == demandas[j] for j in clientes)


m.setObjective(gp.quicksum(transporte[i, j]* custos[i, j] for i in fabricas for j in clientes), sense = gp.GRB.MINIMIZE)
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 13 rows, 36 columns and 72 nonzeros
Model fingerprint: 0xe9a4dfd9
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Presolve time: 0.02s
Presolved: 13 rows, 36 columns, 72 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.7600000e+03   1.100000e+02   0.000000e+00      0s
       5    1.0600000e+04   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.04 seconds
Optimal objective  1.060000000e+04


In [51]:
m.objVal

10600.0

In [54]:
for fabrica in fabricas:
    for cliente in clientes:
        if transporte[fabrica, cliente].x != 0:
            print("{} fornece {} para {}".format(fabrica, transporte[fabrica, cliente].x, cliente))
print("\n")

Fabricas_1 fornece 50.0 para Cliente_1
Fabricas_1 fornece 30.0 para Cliente_4
Fabricas_1 fornece 60.0 para Cliente_9
Fabricas_2 fornece 10.0 para Cliente_3
Fabricas_2 fornece 90.0 para Cliente_6
Fabricas_2 fornece 60.0 para Cliente_9
Fabricas_3 fornece 80.0 para Cliente_2
Fabricas_3 fornece 20.0 para Cliente_3
Fabricas_3 fornece 100.0 para Cliente_5
Fabricas_4 fornece 20.0 para Cliente_4
Fabricas_4 fornece 20.0 para Cliente_7
Fabricas_4 fornece 70.0 para Cliente_8




In [53]:
for fabrica in fabricas:
    print("{}: {} livre".format(fabrica, c_ofertas[fabrica].Slack))

Fabricas_1: 0.0 livre
Fabricas_2: 0.0 livre
Fabricas_3: 0.0 livre
Fabricas_4: 80.0 livre
